In [ ]:
from llama_cpp import Llama
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import re
import os


In [ ]:
# for trials with multiple cohorts (eligible patient subsets), our retrospective trial enrollment dataset
# does not indicate which cohort a patient enrolled on.
# therefore, this notebook uses llama to ascertain if each cohort for each trial on which a patient enrolled was a reasonable cohort consideration.
# this generates a more specific training and evaluation dataset for patient-cohort matching

In [ ]:
llm = Llama.from_pretrained(
    repo_id="lmstudio-community/Meta-Llama-3-70B-Instruct-GGUF",
    filename="*Q4*",
    verbose=False,
    local_dir = '/data/clin_notes_outcomes/meta/',
    cache_dir = '/data/clin_notes_outcomes/meta/',
    main_gpu=1,
    n_ctx=8192,
    n_gpu_layers=-1
)

In [ ]:
# test llama query
response = llm.create_chat_completion(
     messages=[{
         "role": "user",
         "content": "what is the meaning of life?"
     }]
)

In [ ]:
response

In [ ]:
# this function asks llama if a given trial cohort is a reasonable consideration for a given patient summary
def ask_about_trial_loosely(patient_summary, trial_summary, llama_model):
    messages = [{'role':'system', 'content': """You are a brilliant oncologist with encyclopedic knowledge about cancer and its treatment. 
    Your job is to evaluate whether a given clinical trial is a reasonable consideration for a patient, given a clinical trial summary and a patient summary.\n"""}, {'role':'user', 'content': "Here is a summary of the clinical trial:\n" + trial_summary + "\nHere is a summary of the patient:\n" + patient_summary + """
Base your judgment on whether the patient generally fits the cancer type(s), prior treatment(s), and biomarker criteria specified for the trial.
You do not have to determine if the patient is actually eligible; instead please just evaluate whether it is reasonable for the trial to be considered further by the patient's oncologist.
Some trials have biomarker requirements that are not assessed until formal eligibility screening begins; please ignore these requirements.
Reason step by step, then answer the question "Is this trial a reasonable consideration for this patient?" with a one-word Yes! or No! answer."""}]

    outputs = llama_model.create_chat_completion(messages=messages)

    response = outputs['choices'][0]['message']['content']
    if "Yes!" in response:
        eligibility_result = 1.0
    else:
        eligibility_result = 0.0
    
    return outputs, response, eligibility_result
    

In [ ]:
# pull list of retrospective candidate patient-cohort matches for evaluation
cohort_level_trials = pd.read_csv('trial_cohort_lineitems_6-27-24.csv')

In [ ]:
# pull list of retrospective patient summaries
patient_summaries = pd.read_csv('all_patient_summaries_6-27-24.csv')

In [ ]:
patient_summaries.patient_summary.str.contains('no relevant|No relevant').value_counts()

In [ ]:
patient_summaries = patient_summaries[~patient_summaries.patient_summary.str.contains('no relevant|No relevant')]

In [ ]:
cohort_level_trials = cohort_level_trials[['protocol_number','study_nm','protocol_nbr','nct_id','title','brief_summary','detailed_summary','eligibility_criteria','trial_text','cohorts','this_cohort', 'cohort_number']]

In [ ]:
cohort_level_trials = cohort_level_trials[cohort_level_trials.this_cohort.str[0].isin(['1','2','3','4','5','6','7','8','9'])]

In [ ]:
cohort_level_trials.info()

In [ ]:
patient_cohort_candidates = pd.merge(patient_summaries, cohort_level_trials, on='protocol_number')
patient_cohort_candidates.info()

In [ ]:
patient_cohort_candidates['trial_and_cohort'] = patient_cohort_candidates['title'] + "\n" + patient_cohort_candidates['brief_summary'] + "\nEligibility requirements:\n" + patient_cohort_candidates['this_cohort']

In [ ]:
patient_cohort_candidates.info()

In [ ]:
patient_cohort_candidates.trial_and_cohort.iloc[0]

In [ ]:
%%capture
output_list = []
#for i in range(2):
for i in range(501, patient_cohort_candidates.shape[0]):
    output = patient_cohort_candidates.iloc[[i]]

    _, output['llama_response'], output['eligibility_result'] = ask_about_trial_loosely(output['patient_summary'].iloc[0], output['this_cohort'].astype(str).iloc[0], llm)

    output_list.append(output)
    if (i % 500 == 0) or i == (patient_cohort_candidates.shape[0] - 1):
        output_file = pd.concat(output_list, axis=0)
        output_file.to_csv('cohort_specific_eligibility_checks.csv')
